# Import

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,regexp_replace
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
# Inicialize a sessão Spark
spark = SparkSession.builder \
    .appName("ReadS3DataBancos") \
    .getOrCreate()

# Construa o caminho completo para o arquivo no S3
s3_path = f"s3://172684736408-raw/Bancos/*.tsv"
output_s3_path = "s3://172684736408-trusted/Bancos/"

# Leia o arquivo CSV do S3
df_bancos = spark.read.option("encoding", "UTF-8").csv(s3_path, header=True, inferSchema=True, sep='\t')

# Construa o caminho completo para o arquivo no S3
s3_path = f"s3://172684736408-raw/Reclamações/*.csv"
output_s3_path = "s3://172684736408-trusted/Reclamações/"

# Leia o arquivo CSV do S3
df_reclamacoes = spark.read.option("encoding", "UTF-8").csv(s3_path, header=True, inferSchema=True, sep=';')

# Construa o caminho completo para o arquivo no S3
s3_path = f"s3://172684736408-raw/Empregados/*.csv"
output_s3_path = "s3://172684736408-trusted/Empregados/"

# Leia o arquivo CSV do S3
df_empregados = spark.read.option("encoding", "UTF-8").csv(s3_path, header=True, inferSchema=True, sep='|')

In [4]:
#Tratando o campo Nome
df_bancos = df_bancos.withColumn("Nome", regexp_replace(col('Nome'), " - PRUDENCIAL", ""))

#Tratando os nomes
df_reclamacoes = df_reclamacoes.withColumnRenamed("Quantidade total de clientes – CCS e SCR", "Quantidade total de clientes")

#Fazendo as agregações
df_reclamacoes = df_reclamacoes.groupby("Instituição financeira").agg({
        'Índice': 'min',
        'Quantidade total de reclamações': 'sum',
        'Quantidade total de clientes': 'mean'
    })

#Tratando colunas
df_reclamacoes_filtrado = df_reclamacoes.withColumn("Instituição financeira", regexp_replace(col('Instituição financeira'), " \(conglomerado\)", "")) \
                          .withColumnRenamed("Instituição financeira", "Nome") \
                          .withColumnRenamed("min(Índice)", "indice") \
                          .withColumnRenamed("sum(Quantidade total de reclamações)", "qtd_total_reclamacoes") \
                          .withColumnRenamed("avg(Quantidade total de clientes)", "qtd_total_clientes")

#Agrupando e tirando a media
df_empregados = df_empregados.groupBy("Nome").agg({
        'Perspectiva positiva da empresa(%)': 'mean',
        'Remuneração e benefícios': 'mean'
    })
#Renomeando colunas
df_empregados = df_empregados.withColumnRenamed("avg(Perspectiva positiva da empresa(%))", "satisfacao_empregado") \
                          .withColumnRenamed("avg(Remuneração e benefícios)", "satisfacao_salario")

In [1]:
#df_bancos.show()

In [5]:
df_bancos.write.format("parquet").mode("overwrite").option("path", f"{output_s3_path}").save()
df_reclamacoes_filtrado.write.format("parquet").mode("overwrite").option("path", f"{output_s3_path}").save()
df_empregados.write.format("parquet").mode("overwrite").option("path", f"{output_s3_path}").save()

In [6]:
# Encerre a sessão Spark
spark.stop()